# Entrega 2

## Librerias

In [54]:
from rasterio.plot import show as rio_show
from sklearn.preprocessing import MinMaxScaler
from typing import List

import rasterio as rio
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import glob
import json

# Semilla pseudoaleatoria
np.random.seed(24)

# Tamaño de las figuras
plt.rcParams["figure.figsize"] = (20,10)

# Ruta base de la carpeta de datos
DATA_BASE = "/home/ubuntu/deeplearn/deep-learning-e2/datos"

## Exploración

En este apartado vamos a explorar algunas imagenes del conjunto de datos, los archivos que contiene la etiqueta de la imagen y la máscara.

### Carga de imágenes

A continuación vamos a buscar las rutas de acceso a las imagenes. Dado a que el conjunto de etiquetas es más pequeño que las imagenes, vamos a emplear un conjunto reducido tal que tengamos etiquetas para todas la imagenes a emplear

In [4]:
# Directorios de las etiquetas y la imagenes
LABEL_DIR = f"{DATA_BASE}/su_african_crops_ghana_labels"
IMAGE_DIR = f"{DATA_BASE}/su_african_crops_ghana_source_s2"

In [10]:
# Listar los subdirectorios de cada carpeta: Etiquetas
label_dirs = glob.glob(f"{LABEL_DIR}/*")
print(f"Número de etiquetas disponibles: {len(label_dirs)}")

Número de etiquetas disponibles: 4042


In [11]:
# Listar los subdirectorios de cada carpeta: Etiquetas
image_dirs = glob.glob(f"{IMAGE_DIR}/*")
print(f"Número de etiquetas disponibles: {len(image_dirs)}")

Número de etiquetas disponibles: 130212


A continuación vamos a seleccionar solo el subconjunto de las imagenes con etiqueta

In [32]:
def filter_path(path_f: str, label_dirs: List[str]) -> bool:
    """
    Nos permite filtar la ruta de la imagen para determinar
    si esta tiene una máscara asociada.
    """
    try:
        # Filtro para la carpeta de interés
        name = path_f.split("/")[-1]        

        # Filtro para la etiqueta de la imagen
        # El patrón parece ser el siguiente
        # Nombre_source_<Sensor>_<ID>_<Fecha de toma>
        img_id = name.split("_")[6]        

        # Colocar el patrón como se nombran las imágenes
        label_tag = f"{LABEL_DIR}/su_african_crops_ghana_labels_{img_id}"

        # Validar si existe
        return label_tag in label_dirs
    except:
        return False

In [33]:
# Filtrar
image_dirs_2 = [
    tag
    for tag in image_dirs
    if filter_path(path_f=tag, label_dirs=label_dirs)
]

In [34]:
print(f"Imagenes con etiqueta: {len(image_dirs_2)}")

Imagenes con etiqueta: 130210


Como podemos observar, tenemos todas las etiquetas para todas las regiones del dataset. Lo anterior implica que el número de 4042 indica los cultivos disponibles para realizar el análisis. Para mayor facilidad de acceso, vamos a agrupar todas las regiones en una vista que nos permita acceder rápidamente tanto a las imágenes disponibles de la región como a su máscara

In [50]:
# Diccionario para almacenar los datos
# La llave será el ID de la región y su valor serán dos listas:
# La primera con la ruta de las imagenes
# La segunda con la máscara (espero solo sea una)
regions_img_labels = {}

Generar la vista para las máscaras

In [51]:
for label_mask in label_dirs:
    mask_name = label_mask.split("/")[-1]
    mask_id = mask_name.split("_")[-1]
    
    if mask_id not in regions_img_labels:
        # Nueva entrada
        entry_data = {
            "images": [],
            "masks": [label_mask]
        }
        
        # Agregar la entrada al diccionario principal
        regions_img_labels[mask_id] = entry_data
    else:
        # Consultar el diccionario y agregar la otra máscara
        regions_img_labels[mask_id]["masks"].append(label_mask)       

In [52]:
print(f"Regiones disponibles: {len(regions_img_labels.keys())}")

Regiones disponibles: 4042


Generar la vista para las imagenes

In [53]:
for image_path in image_dirs:
    try:        
        # Filtrar el nombre y el ID de la imágen
        image_name = image_path.split("/")[-1]
        image_id = image_name.split("_")[6]

        # Agregar en la lista
        regions_img_labels[image_id]["images"].append(image_path)
    except:
        continue

Vamos a guardar esta vista, para realizar una carga rápida si se requiere

In [56]:
with open(f"{DATA_BASE}/image_view.json", "w", encoding="utf-8") as v_f:
    json.dump(regions_img_labels, v_f, ensure_ascii=False, indent=4)

## Visualización

Ya con la vista lista, vamos a seleccionar de manera aleatoria una imagen para mostrar

In [58]:
view_keys = list(regions_img_labels.keys())
random_idx = np.random.randint(0, len(view_keys))
random_key = view_keys[random_idx]
print(f"Región seleccionada: {random_key}")

Región seleccionada: 000815


Seguidamente vamos a seleccionar la máscara y una de las imágenes

In [61]:
ex_mask_folder = regions_img_labels[random_key]["masks"][0]
ex_mask = f"{ex_mask_folder}/labels.tiff"

In [63]:
ex_img_folder = regions_img_labels[random_key]["images"][0]
ex_img_source = f"{ex_img_folder}/source.tiff"
ex_img_cloudmask = f"{ex_img_folder}/cloudmask.tiff"